In [ ]:
# Dense gas fraction with SMA in CMZoom
# Borrowing from dense_gas_fraction_2018_hatchfield.ipynb
# and Mills_SMA_Share.ipynb

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import aplpy
#from math import *
from astropy import units as u
from astropy.table import Table, Column, MaskedColumn
from astropy.io import ascii
from astropy.wcs import WCS
from reproject import reproject_exact
import re
from astropy.stats import mad_std
import os.path
from matplotlib import collections  as mc
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatter

In [ ]:
# Load in various fits files.

# Figure Path
pathfig='/Users/battersby/Dropbox/Work/cmz/cmzoom_overview/overleaf_cmzoom/10383815hfwsykpqhnrx/figures/'

# Data path
path='/Users/battersby/Dropbox/CMZoom_Data/'

# CMZoom data
cmzoomfits=fits.open(path+'continuum_images/Final_continuum_images/Mosaics/Final/Pb_corrected/'+'CMZoom_continuum_mosaic_no_SgrA.fits')
cmzoom=cmzoomfits[0].data
cmzoomhdr=cmzoomfits[0].header
cmzoomWCS = WCS(cmzoomhdr)

# CMZoom residual
residualfits=fits.open(path+'continuum_images/Final_continuum_images/Mosaics/Final/Pb_corrected/'+'CMZoom_residual_mosaic_no_SgrA.fits')
residual=residualfits[0].data

#CMZoom noise map
sma_noise_fits=fits.open(path+'final_catalog/high_reliability/CMZoom_noisemap_JySr_k14_final.fits')
sma_noise=sma_noise_fits[0].data

# BGPS data
bgpsfits=fits.open(path+'Comparison_Data/'+'BGPS_v2.0_super_gc_13pca_map20_reproject.fits')
bgps=bgpsfits[0].data
bgpshdr=bgpsfits[0].header

# Herschel Data
herschelfits=fits.open(path+'Comparison_Data/'+'column_properunits_conv36_source_only.fits')

# CMZoom Mask
if os.path.exists(path+'dense_gas_fraction/CMZoom_Mask_new_NaNs.fits'):
    maskfits=fits.open(path+'dense_gas_fraction/CMZoom_Mask_new_NaNs.fits')
    mask=maskfits[0].data
    maskhdr=maskfits[0].header
else: 
    maskfits=fits.open(path+'dense_gas_fraction/CMZoom_Mask_new.fits')
    mask=maskfits[0].data
    maskhdr=maskfits[0].header
    mask[mask>50] = float('NaN')
    fits.writeto(path+'dense_gas_fraction/CMZoom_Mask_new_NaNs.fits', data=mask, header=maskhdr, overwrite=True)
#only need to do this once, the freaking 1000s are very annoying
    
#maskfits=fits.open(path+'dense_gas_fraction/CMZoom_Mask_new_NaNs.fits')
#when first made, all nans were 1000s maskfits=fits.open(path+'dense_gas_fraction/CMZoom_Mask_new.fits')
#mask=maskfits[0].data
#maskhdr=maskfits[0].header
#maskWCS=WCS(maskhdr)
#mask[mask>50] = float('NaN')
#only need to do this once, the freaking 1000s are very annoying
#fits.writeto(path+'dense_gas_fraction/CMZoom_Mask_new_NaNs.fits', data=mask, header=maskhdr, overwrite=True)

In [ ]:
# Reproject herschel N(H2) and BGPS data to the same pixel scale as the SMA CMZoom data

if os.path.exists(path+'dense_gas_fraction/Herschel_sma_pix.fits'):
    file = fits.open(path+'dense_gas_fraction/Herschel_sma_pix.fits')
    nh2 = file[0].data
else:
    nh2, footprint = reproject_exact(herschelfits, cmzoomhdr)
    fits.writeto(path+'dense_gas_fraction/Herschel_sma_pix.fits', data=nh2, header=cmzoomhdr, overwrite=True)

if os.path.exists(path+'dense_gas_fraction/BGPS_sma_pix.fits'):
    file = fits.open(path+'dense_gas_fraction/BGPS_sma_pix.fits')
    bgps_sma = file[0].data
else:
    bgps_sma, footprint2 = reproject_exact(bgpsfits, cmzoomhdr)
    fits.writeto(path+'dense_gas_fraction/BGPS_sma_pix.fits', data=bgps_sma, header=cmzoomhdr, overwrite=True)

In [ ]:
# Create a boolean masked array of the CMZoom pixel locations and write to a fits file
# Note that a=b means b follows a! weird. hence, mask_ones=np.copy(mask)

mask_ones = np.copy(mask)
mask_ones[mask_ones < 50] = 1.0

fits.writeto(path+'dense_gas_fraction/Mask_ones.fits', data=mask_ones, header=cmzoomhdr, overwrite=True)

In [ ]:
# Correct for the difference in frequency between BGPS and SMA CMZoom, by 
# Flux density goes as nu^(2+beta)
# (Rayleigh Jean's limit, Snu ~ nu^2 * (1-e^-tau_nu)...
# then optically thin limit, Snu ~ nu^2 * tau_nu...
# tau_nu ~ nu^beta
# So, Snu ~ nu(2+beta))

nu_bgps = 271.1e9 # GHz from Aguirre et al. 2011
nu_sma = 224.9e9 # GHz from SMA bottom LSB 216.9 to top USB 232.9 GHz. Right in the middle.
beta = 1.75 # Battersby et al. 2011

bgps_corr = bgps_sma*mask_ones*((nu_sma/nu_bgps)**(2+beta))

#need to do same for bgps_noise = 45 mJy/beam (average)
bgps_rms = 50.0e-3 * ((nu_sma/nu_bgps)**(2+beta)) #mJy/beam to Jy/beam 
# check Fig. 1 of Ginsburg 2013, about 50 mJy per pixel in the galactic center seems fair

In [ ]:
# BGPS flux should be lower after correction (about a factor of 2).
# This works fine.
print(np.nanmean(bgps_corr))
print(np.nanmean(bgps_sma*mask_ones))
print(bgps_rms)

In [ ]:
# Figure out units of both
print(cmzoomhdr['BUNIT'])
print(bgpsfits[0].header['BUNIT'])
print(sma_noise_fits[0].header['BUNIT'])

In [ ]:
# Convert BGPS from Jy/beam to Jy/pixel
# Jy/pixel = Jy/beam * (beam/sq as) * (sq as / pixel)
# Jy/pixel = Jy/beam * (beam/sq deg) * (sq deg / pixel) # or as here, all in deg^2, so just use that.
bgps_pix = bgps_corr * (1/(bgpshdr['BMAJ']*bgpshdr['BMIN'])) * ((np.absolute(cmzoomhdr['CDELT1'])*np.absolute(cmzoomhdr['CDELT2'])))
cmzoomhdr['BUNIT'] = 'Jy/pixel'
#cmzoomhdr BUNIT IS changed and wrong for old data FYI!
fits.writeto(path+'dense_gas_fraction/BGPS_jypix.fits', data=bgps_pix, header=cmzoomhdr, overwrite=True)
bgps_noise = bgps_rms * (1/(bgpshdr['BMAJ']*bgpshdr['BMIN'])) * ((np.absolute(cmzoomhdr['CDELT1'])*np.absolute(cmzoomhdr['CDELT2'])))

# Convert SMA from Jy/Sr to Jy/pixel for the SMA data, residual, and noise map.
# Jy/pixel = Jy/Sr * Sr/pixel (deg * 2pi rad / 360deg = radian)
sma_pix = np.copy(cmzoom) * (np.absolute(cmzoomhdr['CDELT1'])*np.absolute(cmzoomhdr['CDELT2'])) * (2*np.pi / 360)**2
fits.writeto(path+'dense_gas_fraction/CMZoom_jypix.fits', data=sma_pix, header=cmzoomhdr, overwrite=True)
#residual_pix = residual * (np.absolute(cmzoomhdr['CDELT1'])*np.absolute(cmzoomhdr['CDELT2'])) * (2*np.pi / 360)**2
sma_noise_pix = np.copy(sma_noise) * (np.absolute(cmzoomhdr['CDELT1'])*np.absolute(cmzoomhdr['CDELT2'])) * (2*np.pi / 360)**2

# Get rid of pixels that *exist* in the SMA data map but *not* in the noise map
# (some minor flaw we're not sure how arises, but only a few pixels on the edge)
sma_pix[np.isnan(sma_noise_pix)]=float('NaN')

# makes a *huge* difference what you sum over (everything above 0, or above 1 sigma, or above 2 sigma...)
# Okay, after investigating by eye (10/25/19), I think that counting pixels above 2 sigma is the lowest we can go
# Two sigma cut
sma_pix_2sigma=np.copy(sma_pix)
sma_pix_2sigma[sma_pix_2sigma < (2*sma_noise_pix)]=float('NaN')
bgps_pix_2sigma=np.copy(bgps_pix)
bgps_pix_2sigma[bgps_pix_2sigma < (2*bgps_noise)]=float('NaN')

# One sigma cut
sma_pix_1sigma=np.copy(sma_pix)
sma_pix_1sigma[sma_pix_1sigma < (1*sma_noise_pix)]=float('NaN')
bgps_pix_1sigma=np.copy(bgps_pix)
bgps_pix_1sigma[bgps_pix_1sigma < (1*bgps_noise)]=float('NaN')

# One sigma cut
sma_pix_3sigma=np.copy(sma_pix)
sma_pix_3sigma[sma_pix_3sigma < (3*sma_noise_pix)]=float('NaN')
bgps_pix_3sigma=np.copy(bgps_pix)
bgps_pix_3sigma[bgps_pix_3sigma < (3*bgps_noise)]=float('NaN')


# Make ratio of SMA flux / BGPS flux and save as fits file
#ratio = sma_pix_2sigma/bgps_pix_2sigma
#fits.writeto(path+'dense_gas_fraction/SMAtoBGPS_flux_map.fits', data=ratio, header=cmzoomhdr, overwrite=True)

#Write a bunch of variables to fits file for quick visual exploration to see if the overall DGF makes sense
save_fits=1
if save_fits==1:   
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/sma_pix_2sigma.fits', data=sma_pix_2sigma,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/sma_pix_1sigma.fits', data=sma_pix_1sigma,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/sma_pix_3sigma.fits', data=sma_pix_3sigma,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/bgps_pix_2sigma.fits', data=bgps_pix_2sigma,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/bgps_pix_1sigma.fits', data=bgps_pix_1sigma,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/sma_pix.fits', data=sma_pix,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/bgps_pix.fits', data=bgps_pix,
                 header=cmzoomhdr, overwrite=True)
 #   fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/residual_pix.fits', data=residual_pix,
 #                header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/sma_noise_pix.fits', data=sma_noise_pix,
                 header=cmzoomhdr, overwrite=True)
    fits.writeto(path+'dense_gas_fraction/visual_investigation_of_dgf/bgps_noise.fits', data=np.array(bgps_pix)*0.0+bgps_noise,
                 header=cmzoomhdr, overwrite=True)

In [ ]:
## Get total SMA and BGPS flux within each mask (this only works if mask value is an integer)
# A very IDL way to do things, methinks. Meh, it works! Leave it.

plot_ratio1sig = np.copy(mask) #Make a copy of the mask array to drop values of the SMA/Bolocam ratio into
plot_ratio1sig = 0.0*plot_ratio1sig #Initialize it
S_flux1sig=np.zeros(29) #Initialize other variables
B_flux1sig=np.zeros(29)

plot_ratio2sig = np.copy(mask) #Make a copy of the mask array to drop values of the SMA/Bolocam ratio into
plot_ratio2sig = 0.0*plot_ratio2sig #Initialize it
S_flux2sig=np.zeros(29) #Initialize other variables
B_flux2sig=np.zeros(29)

plot_ratio3sig = np.copy(mask) #Make a copy of the mask array to drop values of the SMA/Bolocam ratio into
plot_ratio3sig = 0.0*plot_ratio2sig #Initialize it
S_flux3sig=np.zeros(29) #Initialize other variables
B_flux3sig=np.zeros(29)
Hnh2med=np.zeros(29)

for i in np.arange(0,29): # Loop through each mask
    S_flux1sig[i] = np.nansum(sma_pix_1sigma[mask==i+1]) # Sum SMA above pre-defined sigma level
    B_flux1sig[i] = np.nansum(bgps_pix_1sigma[mask==i+1]) # Sum bolocam above pre-defined sigma level
    plot_ratio1sig[(mask==i+1)] = (S_flux1sig[i]/B_flux1sig[i]) #Fill each mask with the ratio of SMA to Bolocam
    
    S_flux2sig[i] = np.nansum(sma_pix_2sigma[mask==i+1]) # Sum SMA above pre-defined sigma level
    B_flux2sig[i] = np.nansum(bgps_pix_2sigma[mask==i+1]) # Sum bolocam above pre-defined sigma level
    plot_ratio2sig[(mask==i+1)] = (S_flux2sig[i]/B_flux2sig[i]) #Fill each mask with the ratio of SMA to Bolocam
    
    S_flux3sig[i] = np.nansum(sma_pix_3sigma[mask==i+1]) # Sum SMA above pre-defined sigma level
    B_flux3sig[i] = np.nansum(bgps_pix_3sigma[mask==i+1]) # Sum bolocam above pre-defined sigma level
    plot_ratio3sig[(mask==i+1)] = (S_flux3sig[i]/B_flux3sig[i]) #Fill each mask with the ratio of SMA to Bolocam
    Hnh2med[i]=np.nanmedian(nh2[mask==i+1])

In [ ]:
print(np.nanmin(plot_ratio1sig), np.nanmax(plot_ratio1sig), np.nanmedian(plot_ratio1sig))
print(np.nanmin(plot_ratio2sig), np.nanmax(plot_ratio2sig), np.nanmedian(plot_ratio2sig))
print(np.nanmin(plot_ratio3sig), np.nanmax(plot_ratio3sig), np.nanmedian(plot_ratio3sig))
#print(Hnh2med)

In [ ]:
# Perry - this is the last cell to run right now!
# Plot the ratio of SMA flux to BGPS flux  (remember, highly dependent on sigma cut!)
%matplotlib nbagg 

plt.rcParams.update({'font.size': 12}) #set fontsize

fig, ax = plt.subplots(figsize=(12, 8))
ax = plt.subplot(projection=cmzoomWCS)

#Plot 3 sigma with colorbar and 2 sigma without so I can make a mutliplot in latex
cax = ax.imshow(plot_ratio3sig, norm=LogNorm(vmin=0.01, vmax=0.5), cmap='viridis')

plt.text(+1.5, -0.05, 'All pixels above 3 $\sigma$', color='black')#,alpha=alpha) 

#ax.set_title('Dense Gas Fraction, SMA Flux / BGPS Flux')
ax.set_xlabel('Galactic Longitude')
ax.set_ylabel('Galactic Latitude')

#Plot ticks as I choose, rather than the base only
# (from: https://stackoverflow.com/questions/27345005/log-labels-on-colorbar-matplotlib)
formatter = LogFormatter(10, labelOnlyBase=False) 
cbar = fig.colorbar(cax, ticks=[0.05,0.1, 0.2, 0.3, 0.4, 0.5, 0.75, 1.0], format=formatter, orientation='horizontal')
cbar.ax.set_xticklabels(['5%','10%','20%','30%','40%','50%','75%','100%'])
cbar.ax.set_title('Dense Gas Fraction = SMA Flux / BGPS Flux')# Above 2 $\sigma$')

plt.show()
plt.savefig(pathfig+'CMZoom_dense_gas_fraction_3sigma.pdf',
            format='pdf', dpi=500, bbox_inches='tight')#, pad_inches=0.1)


In [ ]:
S_flux = S_flux3sig
B_flux = B_flux3sig
# Write to a table
clouds=np.arange(0,29)
data = Table([np.arange(1,30),S_flux, B_flux, Hnh2med, S_flux/B_flux], 
             names=['cloud number', 'Total SMA Flux', 'Total BGPS Flux', 'Column Density', 'Ratio'])

ascii.write(data,path+'dense_gas_fraction/values.dat', overwrite=True)
             

In [ ]:
# Now dense gas fraction per column density bin.
# Calculate total mass of each cloud in Herschel
# Then calculate the fraction of total SMA mass in column density bins
# from 5x10^22 - 5x10^24 cm^-2
# (but what does this mean for a point source, which would have some % low density and some % high density bc of beam?)


In [ ]:
# Constants (and dust property assumptions) for
# conversion from flux to column and mass

# Assumptions
T = 20. #15.
kappa = 0.01*4.0*(nu_sma/505e9)**1.75 # cm^2/g
#kappa above from Battersby et al. 2011 (pg 6), but multiplied by 0.01 
#for dust to gas ratio of 100 
distance = 8340 #pc Reid 2014
dm = distance*3.086e16 #distance in m
dcm = distance*3.086e18 # distance in cm

# Constants
c = 2.9979e8 # m/s
h = 6.626e-34
nu_sma = 224.9e9 #Hz central frequency of SMA observations
mu = 2.8 # Kauffman et al. 2008 appendix (mean molecular weight of molecular gas)
mh = 1.67e-27 # kg
msun = 1.99e30 #kg
k = 1.38e-23 #SI

In [ ]:
## Define Blackbody Function
#(SI units)
def BBody(nu,T):
    return (2*h*(nu_sma**3))/((c**2)*(np.exp((h*nu)/(k*T))-1.))

## Convert the flux in Jy to a column density in cm^-2
# Assuming dust properties (kappa, mu, nu, T).
# This assumes optically thin dust. Snu = BBody*(1-e^-tau_nu), tau_nu = mu*mh*kappa*N(H2)
# So, N(H2) = Snu / (BB * mu * mh * kappa). Snu and BB should match units. 
# BB is SI fully and S_Jy is too, but off by 1e-26 (1 Jy = 1e-26 J/ (s * m^2 * Hz)) - so multiply S_Jy by 1e-26
# Then kappa is cm^2/g and mh is kg, so multiply kappa by 1000 to get just cm^-2 in the end
#S is in Jy/Sr, nu in Hz, T in K, and column in cm^-2
def findColumn(S_Jy, nu, T):
    return ((S_Jy*1e-26)/(kappa*1000.*mu*mh*BBody(nu,T)))

# Now convert column density to mass
#column is in cm^-2, pix_area_cm is the pixel area in cm^2, and mass is solar masses
def findMass(column, pix_area_cm2):
    return (column * mh * mu * pix_area_cm2 / msun)

#def getmasshph(S_Jy, nu, T, D):
#    return ((S_Jy/1e26)*D*D)/(kappa*BBody(nu,T))


In [ ]:
### Convert Herschel N(H2) (number of H2 atoms per cm^-2) to Msun/pix

# Get pixel area in square centimeter
pixx_cm = dcm*(np.absolute(cmzoomhdr['CDELT1'])*3600./206265) # distance in cm * angle in radians
pixy_cm = dcm*(np.absolute(cmzoomhdr['CDELT2'])*3600./206265)
pix_area_cm2 = pixx_cm*pixy_cm

# Convert N(H2) to Mass
mass_herschel = findMass(nh2, pix_area_cm2)
fits.writeto(path+'dense_gas_fraction/Herschel_Msun_per_pix.fits', data=mass_herschel, header=cmzoomhdr, overwrite=True)

In [ ]:
# Convert SMA flux to Msun/pix
#(pix area is the same herschel, since we re-gridded them)
# sma_pix is the SMA flux in units of Jy/pixel
# and cmzoom is the SMA flux in units of Jy/Sr
# ##residual is the SMA noise in units of Jy/Sr
# nope, now sma_noise is the SMA noise in units of Jy/Sr and sma_noise_pix is in units of Jy/pixel
# residual_pix is the SMA noise in units of Jy/pixel

sma_column = findColumn(cmzoom, nu_sma, T)
sma_mass = findMass(sma_column, pix_area_cm2)
#sma_noise_column = findColumn(residual, nu_sma, T)
sma_noise_column = findColumn(sma_noise, nu_sma, T)
sma_noise_mass = findMass(sma_noise_column, pix_area_cm2)
sma_cloud_noise_column = findColumn(sma_noise, nu_sma,T)

# Only take pixels above 3 sigma
sma_column[sma_column <= (3*sma_noise_column)]=float('NaN')
sma_mass[sma_column <= (3*sma_noise_column)]=float('NaN')

fits.writeto(path+'dense_gas_fraction/SMA_Msun_per_pix.fits', data=sma_mass, header=cmzoomhdr, overwrite=True)
fits.writeto(path+'dense_gas_fraction/SMA_NH2_per_pix.fits', data=sma_column, header=cmzoomhdr, overwrite=True)
fits.writeto(path+'dense_gas_fraction/SMA_Msun_Herschel_Msun_per_pix.fits', data=sma_mass/mass_herschel, 
            header=cmzoomhdr, overwrite=True)
fits.writeto(path+'dense_gas_fraction/SMA_noise_NH2_pix.fits', data=sma_noise_column, 
            header=cmzoomhdr, overwrite=True)

In [ ]:
# SMA Mass Percent = smp ==> Percentage of SMA mass above this column density / Herschel Mass 
# smpsma ==> Percentage of SMA mass above this column density / Total SMA Mass
# sma_herschel ==> Percentage of total SMA mass / Total Herschel Mass
# Units of nh2 = 1e22 cm^-2, so smp5 = 5x10^22, smp50 = 5x10^23

def getSMP(sma_column, mass_herschel, sma_mass, factor):
    smp = np.zeros(29)#np.copy(mask)
    smacloudmass = np.zeros(29)
    mtot = np.zeros(29)
    sma_tot = np.zeros(29)
    smpsma = np.zeros(29)
    sma_herschel = np.copy(mask)
    sma_herschel=0.0*sma_herschel
    for i in np.arange(0,29):
        smacloudmass[i] = np.nansum(sma_mass[(mask==i+1)*(sma_column >= factor*1e22)])
        sma_tot[i] = np.nansum(sma_mass[(mask==i+1)*(sma_column > 0)])
        mtot[i] = np.nansum(mass_herschel[(mask==i+1)*(sma_column > 0)]) # should it be herschel mass only where sma>0?
        smp[i] = (smacloudmass[i]/mtot[i])
        smpsma[i] = (smacloudmass[i]/sma_tot[i])
        sma_herschel[mask==i+1] = (sma_tot[i]/mtot[i])
    return smp, smpsma, sma_herschel, sma_tot, mtot

smp0, smpsma0, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 0.0)#NOT USEFUL BC NOISE?!
smp1, smpsma1, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 1.0)
smp2, smpsma2, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 2.0)
smp3, smpsma3, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 3.0)
smp4, smpsma4, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 4.0)
smp5, smpsma5, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 5.0)
smp10, smpsma10, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 10.0)
smp15, smpsma15, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 15.0)
smp20, smpsma20, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 20.0)
smp25, smpsma25, sma_herschel, sma_tot, mtot = getSMP(sma_column, mass_herschel, sma_mass, 25.0)


In [ ]:
# Get smp and smpsma for overall survey
smp0

In [ ]:
# Plot the column density fractions
%matplotlib nbagg

#plt.style.use('battersbot-colorblind')
plt.style.use('seaborn-colorblind')
#plt.style.use('classic')
#plt.style.use('fivethirtyeight')

plt.rcParams.update({'font.size': 18}) #set fontsize
fig = plt.figure(1,figsize=(11,6))
ax = fig.gca()
#ax.set_xscale('log')

# things to plot: np.arange(1,30),S_flux, B_flux, Hnh2med, S_flux/B_flux], smp, smpsma, sma_herschel, sma_tot, mtot

#plt.plot(Hnh2med, S_flux/B_flux,'o',label=r'SMA Flux / BGPS Flux')
plt.plot(Hnh2med, smp1,'o', label=r'SMA Mass Above 1e22 / Herschel Mass')
plt.plot(Hnh2med, smp5,'o', label=r'SMA Mass Above 5e22 / Herschel Mass')
plt.plot(Hnh2med, smp10,'o', label=r'SMA Mass Above 10e22 / Herschel Mass')
plt.plot(Hnh2med, smp15,'o', label=r'SMA Mass Above 15e22 / Herschel Mass')
#plt.plot(Hnh2med, smpsma1,'o', label=r'SMA Mass Above 1e22 / SMA Mass')


#plt.plot([0,0], [0,0], lw=2) #, label=r'Herschel, with BG', ls=':')
#plt.plot([0,0], [0,0], lw=2) #,label=r'KKC17 Simulation', ls='--')
#plt.plot(l,m500pc, lw=2, label=r'Ridley+ Simulation, with chemistry')#, ls='--')
#plt.plot(l_old,m500pc_old, lw=2, label=r'Ridley+ Simulation')#, ls='--')

plt.ylabel('Dense Gas Fraction')
plt.xlabel(r'Median Herschel N(H$_2$)')
ax.set_ylim([0.01,1])
ax.set_xlim([1.5e22, 3.5e23])
ax.set_yscale('log')
#plt.gca().invert_xaxis()
plt.gcf().subplots_adjust(bottom=0.15) # make room for x-axis

legend = plt.legend(loc='upper right', shadow=True, fontsize=14)#'x-large')

plt.savefig(pathfig+'CMZoom_sma_mass_vs_herschel_fraction.pdf',
            format='pdf', dpi=500, bbox_inches='tight')


In [ ]:
# Plot the column density fractions
%matplotlib nbagg

#plt.style.use('battersbot-colorblind')
plt.style.use('seaborn-colorblind')
#plt.style.use('classic')
#plt.style.use('fivethirtyeight')

plt.rcParams.update({'font.size': 18}) #set fontsize
fig = plt.figure(1,figsize=(11,6))
ax = fig.gca()
#ax.set_yscale('log')

# things to plot: np.arange(1,30),S_flux, B_flux, Hnh2med, S_flux/B_flux], smp, smpsma, sma_herschel, sma_tot, mtot

plt.plot(Hnh2med, S_flux/B_flux,'o',label=r'SMA Flux / BGPS Flux', alpha=0.65)
plt.plot(Hnh2med, smpsma1,'d', label=r'SMA Mass Above 1$\times 10^{22}$ / Total', alpha=0.65)
plt.plot(Hnh2med, smpsma5,'*', label=r'SMA Mass Above 5$\times 10^{22}$ / Total', alpha=0.65)
plt.plot(Hnh2med, smpsma10,'P', label=r'SMA Mass Above 1$\times 10^{23}$ / Total', alpha=0.65)
plt.plot(Hnh2med, smpsma15,'X', label=r'SMA Mass Above 1.5$\times 10^{23}$ / Total', alpha=0.65)

#plt.plot([0,0], [0,0], lw=2) #, label=r'Herschel, with BG', ls=':')
#plt.plot([0,0], [0,0], lw=2) #,label=r'KKC17 Simulation', ls='--')
#plt.plot(l,m500pc, lw=2, label=r'Ridley+ Simulation, with chemistry')#, ls='--')
#plt.plot(l_old,m500pc_old, lw=2, label=r'Ridley+ Simulation')#, ls='--')

plt.ylabel('Dense Gas Fraction')
plt.xlabel(r'Median Herschel N(H$_2$) [cm$^{-2}$]')
ax.set_ylim([0,1])
ax.set_xlim([1.5e22, 3.5e23])
#plt.gca().invert_xaxis()
plt.gcf().subplots_adjust(bottom=0.15) # make room for x-axis

legend = plt.legend(loc='upper right', shadow=True, fontsize=14)#'x-large')

plt.savefig(pathfig+'CMZoom_sma_mass_fraction.pdf',
            format='pdf', dpi=500, bbox_inches='tight')


In [ ]:
# Plot the column density fractions of SMA mass / total SMA mass as a BAR Graph
#reminder: 
# SMA Mass Percent = smp ==> Percentage of SMA mass above this column density / Herschel Mass 
# smpsma ==> Percentage of SMA mass above this column density / Total SMA Mass
# sma_herschel ==> Percentage of total SMA mass / Total Herschel Mass

%matplotlib nbagg

#plt.style.use('battersbot-colorblind')
plt.style.use('classic')
plt.style.use('seaborn-colorblind')
#
#plt.style.use('fivethirtyeight')

#import seaborn as sns
#sns.set()
#current_palette = sns.color_palette()
#sns.palplot(sns.color_palette("Blues"))

plt.rcParams.update({'font.size': 18}) #set fontsize
fig = plt.figure(1,figsize=(11,6))
ax = fig.gca()
#ax.set_yscale('log')

# things to plot: np.arange(1,30),S_flux, B_flux, Hnh2med, S_flux/B_flux], smp, smpsma, sma_herschel, sma_tot, mtot

#configs = dataset[0]
N = 29
ind = np.arange(N)
width = 1

#p1 = plt.bar(ind, dataset[1], width, color='r')
#p2 = plt.bar(ind, dataset[2], width, bottom=dataset[1], color='b')
#p3 = plt.bar(ind, dataset[3], width, 
#             bottom=np.array(dataset[1])+np.array(dataset[2]), color='g')
#p4 = plt.bar(ind, dataset[4], width,
#             bottom=np.array(dataset[1])+np.array(dataset[2])+np.array(dataset[3]),
#             color='c')

plt.bar(ind, smpsma0, width, color='lightgray')
plt.bar(ind, smpsma1, width)
plt.bar(ind, smpsma5, width)
plt.bar(ind, smpsma10, width)
#plt.bar(ind, smpsma15, width)
plt.bar(ind, smpsma20, width)
#plt.bar(ind, smpsma25, width)


plt.ylabel('Dense Gas Fraction')
#plt.xlabel(r'Median Herschel N(H$_2$) [cm$^{-2}$]')
plt.xlabel(r'Cloud Number')
ax.set_ylim([0,1])
ax.set_xlim([0,29])
#ax.set_xlim([1.5e22, 3.5e23])
#plt.gca().invert_xaxis()
plt.gcf().subplots_adjust(bottom=0.15) # make room for x-axis

plt.legend(['SMA Mass < 1 x $10^{22}$ cm$^{-2}$','SMA Mass > 1 x $10^{22}$ cm$^{-2}$', 'SMA Mass > 5 x $10^{22}$ cm$^{-2}$', 
            'SMA Mass > 1 x $10^{23}$ cm$^{-2}$', 'SMA Mass > 2 x $10^{23}$ cm$^{-2}$'],
           loc=2, fontsize=10)

plt.savefig(pathfig+'CMZoom_sma_mass_fraction_bar.pdf',
            format='pdf', dpi=500, bbox_inches='tight')


In [ ]:
# Plot the column density fractions of SMA mass / Herschel mass as a BAR Graph
# With Cloud name instead of number
%matplotlib nbagg
#reminder: 
# SMA Mass Percent = smp ==> Percentage of SMA mass above this column density / Herschel Mass 
# smpsma ==> Percentage of SMA mass above this column density / Total SMA Mass
# sma_herschel ==> Percentage of total SMA mass / Total Herschel Mass

#plt.style.use('battersbot-colorblind')
plt.style.use('classic')
plt.style.use('seaborn-colorblind')
#
#plt.style.use('fivethirtyeight')

#import seaborn as sns
#sns.set()
#current_palette = sns.color_palette()
#sns.palplot(sns.color_palette("Blues"))

plt.rcParams.update({'font.size': 18}) #set fontsize
fig = plt.figure(1,figsize=(11,6))
ax = fig.gca()
ax.set_yscale('log')

# things to plot: np.arange(1,30),S_flux, B_flux, Hnh2med, S_flux/B_flux], smp, smpsma, sma_herschel, sma_tot, mtot

#configs = dataset[0]
N = 29
ind = np.arange(N)
width = 1

#p1 = plt.bar(ind, dataset[1], width, color='r')
#p2 = plt.bar(ind, dataset[2], width, bottom=dataset[1], color='b')
#p3 = plt.bar(ind, dataset[3], width, 
#             bottom=np.array(dataset[1])+np.array(dataset[2]), color='g')
#p4 = plt.bar(ind, dataset[4], width,
#             bottom=np.array(dataset[1])+np.array(dataset[2])+np.array(dataset[3]),
#             color='c')

#plt.bar(ind, smp0, width, color='lightgray')
plt.bar(ind, smp1, width)
plt.bar(ind, smp2, width)
plt.bar(ind, smp3, width)
plt.bar(ind, smp4, width)
plt.bar(ind, smp5, width)
plt.bar(ind, smp10, width)
#plt.bar(ind, smp15, width)
#plt.bar(ind, smp20, width, color='gray')
#plt.bar(ind, smp25, width)


x=ind
my_xticks = ['a','b','c','d','e','f','g','h', 'i', 'j','k','l','m','n','o','p','q',
             'r','s','t','u','v','w','x','y','z','aa','bb','cc']
plt.xticks(x, my_xticks)
#plt.plot(x, y)
#plt.show()

plt.ylabel('SMA Mass/Total Herschel Mass')
#plt.xlabel(r'Median Herschel N(H$_2$) [cm$^{-2}$]')
plt.xlabel(r'Cloud Number')
ax.set_ylim([0.01,1])
ax.set_xlim([0,29])
#ax.set_xlim([1.5e22, 3.5e23])
#plt.gca().invert_xaxis()
plt.gcf().subplots_adjust(bottom=0.15) # make room for x-axis

#legend = plt.legend(loc='upper right', shadow=True, fontsize=14)#'x-large')

plt.legend(['SMA Mass > 1 x $10^{22}$ cm$^{-2}$', 'SMA Mass > 2 x $10^{22}$ cm$^{-2}$',
            'SMA Mass > 3 x $10^{22}$ cm$^{-2}$','SMA Mass > 4 x $10^{22}$ cm$^{-2}$',
            'SMA Mass > 5 x $10^{22}$ cm$^{-2}$', 
            'SMA Mass > 1 x $10^{23}$ cm$^{-2}$'],
           loc='upper left', fontsize=10)
#, 'SMA Mass > 2 x $10^{23}$ cm$^{-2}$'
plt.savefig(pathfig+'CMZoom_sma_mass_vs_herschel_fraction_bar.pdf',
            format='pdf', dpi=500, bbox_inches='tight')


In [ ]:
# Plot CMZoom Mask with  labels on top. Labels and such from "CMZoom_Coverage.ipynb"
# Plot SMA mask and assign cloud names to the numbers
# with labels is obscenely complex! There must be a better way...?
%matplotlib nbagg

#Set to 1 if you want to create the coverage file with cloud labels, 0 if you want the one without
# why so complex?!
with_labels = 0

plt.rcParams.update({'font.size': 14}) #set fontsize

fig=aplpy.FITSFigure(path+'dense_gas_fraction/CMZoom_Mask_new_NaNs.fits', convention='wells')
#fig=aplpy.FITSFigure(path2+'Continuum_mosaic.fits', convention='wells')
fig.recenter(0.5,0.0, width=2.8, height=0.7)
#fig.show_colorscale(cmap='viridis', vmin=0, vmax=30)
#fig.show_colorscale(cmap='tab20c')#, vmin=0, vmax=30)


from matplotlib.colors import ListedColormap
import math

def generate_colormap(N):
    arr = np.arange(N)/N
    N_up = int(math.ceil(N/7)*7)
    arr.resize(N_up)
    arr = arr.reshape(7,N_up//7).T.reshape(-1)
    ret = plt.cm.hsv(arr)
    n = ret[:,3].size
    a = n//2
    b = n-a
    for i in range(3):
        ret[0:n//2,i] *= np.arange(0.2,1,0.8/a)
    ret[n//2:,3] *= np.arange(1,0.1,-0.9/b)
    return ret

N = 16
H = np.arange(N*N).reshape([N,N])

fig.show_colorscale(cmap=ListedColormap(generate_colormap(N*N)))#, vmin=0, vmax=30)

#fig.show_colorscale(cmap='Purples', vmin=0.1e+22,vmax=2e+23)
fig.show_colorbar()
fig.colorbar.set_width(0.3)

#fig.set_theme('publication')
fig.set_tick_labels_format(xformat='ddd.d', yformat='dd.d')
#fig.set_theme('publication')

fig.add_scalebar(0.674*u.degree, color='black',alpha=0.65)#, fontsize=6)
fig.scalebar.set_corner('top left')
fig.scalebar.set_label('100 pc')#, fontsize=14)

#fig.show_contour(path+'CMZoom_Mask_v1.fits',linewidths='1',colors='black', alpha=0.65, levels=[0], convention='wells')
plt.gcf().subplots_adjust(bottom=0.2) # make room for x-axis
plt.gcf().subplots_adjust(left=0.2) # make room for y-axis

if with_labels==0:
    #fig.add_label(-0.5,0.25, 'CMZoom Coverage', color='black',alpha=0.65, fontsize=10)
    plt.savefig(pathfig+'CMZoom_mask_nolabels.pdf',format='pdf', dpi=500, bbox_inches='tight')#, pad_inches=0.1)
else:
    plt.rcParams.update({'font.size': 8}) #set fontsize
    color='black'
    alpha=0.65
    cloud16 = np.array([[1.65,1.65],[-0.22,-0.17]])
    cloud11 = np.array([[0.98,0.98],[-0.2,-0.1]])
    missing = np.array([[0.8,0.8],[-0.30,-0.25]])
    sgrb2ext = np.array([[0.72,0.72],[-0.27,-0.20]])
    sgrb2 = np.array([[0.69,0.69],[0.13,0.06]])
    sgrb2nw = np.array([[0.65,0.65],[0.16,0.08]])
    #dref = np.array([[0.49,0.49],[-0.1,-0.04]])
    dref = np.array([[0.49,0.49],[0.24,0.1]])
    drdcb = np.array([[0.38,0.38],[0.24, 0.1]])
    far0 = np.array([[0.36,0.36],[-0.14,-0.11]])
    brick = np.array([[0.25,0.25],[0.1,0.05]])
    tlp = np.array([[0.11,0.11],[-0.29, -0.12]])
    fifty = np.array([[-0.008,-0.008],[-0.16,-0.12]])
    twenty = np.array([[359.88,359.88],[-0.22,-0.115]])
    cnd = np.array([[359.94,359.94],[0.05,0.001]])
    bridge = np.array([[0.09,0.09],[0.22,0.02]])
    arches = np.array([[0.01,0.01],[0.14,0.05]])
    arches2 = np.array([[0.05,0.05],[0.14,0.05]])
    far1 = np.array([[359.86,359.86],[0.2,0.05]])
    far2 = np.array([[359.73,359.73],[0.2,0.05]])
    far3 = np.array([[359.61,359.61],[0.2,0.05]])
    sgrc = np.array([[359.46,359.46],[-0.02,-0.08]])
    stream = np.array([[359.65,359.65],[-0.29,-0.17]])
    isolated = np.array([[359.14,359.14],[-0.34,0.0]])
    line_list=[cloud16,cloud11,missing,sgrb2ext,sgrb2,sgrb2nw,dref,drdcb,far0,brick,tlp,
               fifty,twenty,cnd,bridge,arches,arches2,far1,far2,far3,sgrc,stream,isolated]
    fig.show_lines(line_list, color='black',alpha=0.65)
    fig.add_label(cloud16[0,0],cloud16[1,0]-0.03, '1.6$^{\circ}$cloud', color=color,alpha=alpha)
    fig.add_label(cloud11[0,0],cloud11[1,0]-0.03, '1.1$^{\circ}$cloud', color=color,alpha=alpha)
    fig.add_label(missing[0,0],missing[1,0]-0.03, 'Not covered', color=color,alpha=alpha)
    fig.add_label(sgrb2ext[0,0]-0.13,sgrb2ext[1,0]+0.02, 'SgrB2 Ext', color=color,alpha=alpha)
    fig.add_label(sgrb2[0,0]+0.08,sgrb2[1,0]-0.02, 'SgrB2', color=color,alpha=alpha)
    fig.add_label(sgrb2nw[0,0],sgrb2nw[1,0]+0.03, 'SgrB2 NW', color=color,alpha=alpha)
    #fig.add_label(dref[0,0],dref[1,0]-0.05, 'Dust Ridge e/f', color=color,alpha=alpha)
    fig.add_label(drdcb[0,0]+0.02,drdcb[1,0]+0.07, 'Dust Ridge', color=color,alpha=alpha)
    fig.add_label(dref[0,0],dref[1,0]+0.03, 'e/f', color=color,alpha=alpha)
    fig.add_label(drdcb[0,0],drdcb[1,0]+0.03, 'd,c,b', color=color,alpha=alpha)
    fig.add_label(far0[0,0],far0[1,0]-0.03, 'FSC', color=color,alpha=alpha)
    #fig.add_label(far0[0,0]+0.04,far0[1,0]-0.05, 'candidate', color=color,alpha=alpha)
    fig.add_label(brick[0,0],brick[1,0]+0.03, 'Brick', color=color,alpha=alpha)
    fig.add_label(tlp[0,0],tlp[1,0]-0.03, 'Three Little Pigs', color=color,alpha=alpha)
    fig.add_label(fifty[0,0],fifty[1,0]-0.03, '50kms$^{-1}$', color=color,alpha=alpha)
    fig.add_label(twenty[0,0],twenty[1,0]-0.03, '20kms$^{-1}$', color=color,alpha=alpha)
    fig.add_label(cnd[0,0],cnd[1,0]+0.03, 'CND', color=color,alpha=alpha)
    fig.add_label(bridge[0,0],bridge[1,0]+0.03, 'Bridge', color=color,alpha=alpha) #H$_2$CO 
    fig.add_label(arches[0,0]-0.01,arches[1,0]+0.03, 'Arches', color=color,alpha=alpha) 
    fig.add_label(far2[0,0]-0.07,far2[1,0]+0.03, 'Far-side candidates (FSCs)', color=color,alpha=alpha) 
    fig.add_label(sgrc[0,0],sgrc[1,0]+0.03, 'SgrC', color=color,alpha=alpha) 
    fig.add_label(stream[0,0],stream[1,0]-0.03, 'Stream', color=color,alpha=alpha) 
    fig.add_label(isolated[0,0]+0.19,isolated[1,0]+0.1, 'Five circles are', color=color,alpha=alpha) 
    fig.add_label(isolated[0,0]+0.18,isolated[1,0]+0.06, 'isolated HMSF', color=color,alpha=alpha) 
    fig.add_label(isolated[0,0]+0.14,isolated[1,0]+0.02, 'candidates', color=color,alpha=alpha) 

    plt.savefig(pathfig+'CMZoom_mask.pdf',format='pdf', dpi=500, bbox_inches='tight')#, pad_inches=0.1)
    
plt.show()




In [ ]:
### BACKUP OF THE COLUMN DENSITY PLOT WITH CLOUD NUMBER INSTEAD OF NAME
# Plot the column density fractions of SMA mass / Herschel mass as a BAR Graph
%matplotlib nbagg
#reminder: 
# SMA Mass Percent = smp ==> Percentage of SMA mass above this column density / Herschel Mass 
# smpsma ==> Percentage of SMA mass above this column density / Total SMA Mass
# sma_herschel ==> Percentage of total SMA mass / Total Herschel Mass

#plt.style.use('battersbot-colorblind')
plt.style.use('classic')
plt.style.use('seaborn-colorblind')
#
#plt.style.use('fivethirtyeight')

#import seaborn as sns
#sns.set()
#current_palette = sns.color_palette()
#sns.palplot(sns.color_palette("Blues"))

plt.rcParams.update({'font.size': 18}) #set fontsize
fig = plt.figure(1,figsize=(11,6))
ax = fig.gca()
ax.set_yscale('log')

# things to plot: np.arange(1,30),S_flux, B_flux, Hnh2med, S_flux/B_flux], smp, smpsma, sma_herschel, sma_tot, mtot

#configs = dataset[0]
N = 29
ind = np.arange(N)
width = 1

#p1 = plt.bar(ind, dataset[1], width, color='r')
#p2 = plt.bar(ind, dataset[2], width, bottom=dataset[1], color='b')
#p3 = plt.bar(ind, dataset[3], width, 
#             bottom=np.array(dataset[1])+np.array(dataset[2]), color='g')
#p4 = plt.bar(ind, dataset[4], width,
#             bottom=np.array(dataset[1])+np.array(dataset[2])+np.array(dataset[3]),
#             color='c')

#plt.bar(ind, smp0, width, color='lightgray')
plt.bar(ind, smp1, width)
plt.bar(ind, smp2, width)
plt.bar(ind, smp3, width)
plt.bar(ind, smp4, width)
plt.bar(ind, smp5, width)
plt.bar(ind, smp10, width)
#plt.bar(ind, smp15, width)
#plt.bar(ind, smp20, width, color='gray')
#plt.bar(ind, smp25, width)

plt.ylabel('SMA Mass/Total Herschel Mass')
#plt.xlabel(r'Median Herschel N(H$_2$) [cm$^{-2}$]')
plt.xlabel(r'Cloud Number')
ax.set_ylim([0.01,1])
ax.set_xlim([0,29])
#ax.set_xlim([1.5e22, 3.5e23])
#plt.gca().invert_xaxis()
plt.gcf().subplots_adjust(bottom=0.15) # make room for x-axis

#legend = plt.legend(loc='upper right', shadow=True, fontsize=14)#'x-large')

plt.legend(['SMA Mass > 1 x $10^{22}$ cm$^{-2}$', 'SMA Mass > 2 x $10^{22}$ cm$^{-2}$',
            'SMA Mass > 3 x $10^{22}$ cm$^{-2}$','SMA Mass > 4 x $10^{22}$ cm$^{-2}$',
            'SMA Mass > 5 x $10^{22}$ cm$^{-2}$', 
            'SMA Mass > 1 x $10^{23}$ cm$^{-2}$'],
           loc='upper left', fontsize=10)
#, 'SMA Mass > 2 x $10^{23}$ cm$^{-2}$'
plt.savefig(pathfig+'CMZoom_sma_mass_vs_herschel_fraction_bar.pdf',
            format='pdf', dpi=500, bbox_inches='tight')


In [ ]:
#IGNORE FOR NOW
cloud16 = np.array([[1.65,1.65],[-0.22,-0.17]])
cloud11 = np.array([[0.98,0.98],[-0.2,-0.1]])
missing = np.array([[0.8,0.8],[-0.30,-0.25]])
sgrb2ext = np.array([[0.72,0.72],[-0.27,-0.20]])
sgrb2 = np.array([[0.69,0.69],[0.13,0.06]])
sgrb2nw = np.array([[0.65,0.65],[0.16,0.08]])
#dref = np.array([[0.49,0.49],[-0.1,-0.04]])
dref = np.array([[0.49,0.49],[0.24,0.1]])
drdcb = np.array([[0.38,0.38],[0.24, 0.1]])
far0 = np.array([[0.36,0.36],[-0.14,-0.11]])
brick = np.array([[0.25,0.25],[0.1,0.05]])
tlp = np.array([[0.11,0.11],[-0.29, -0.12]])
fifty = np.array([[-0.008,-0.008],[-0.16,-0.12]])
twenty = np.array([[359.88,359.88],[-0.22,-0.115]])
cnd = np.array([[359.94,359.94],[0.05,0.001]])
bridge = np.array([[0.09,0.09],[0.22,0.02]])
arches = np.array([[0.01,0.01],[0.14,0.05]])
arches2 = np.array([[0.05,0.05],[0.14,0.05]])
far1 = np.array([[359.86,359.86],[0.2,0.05]])
far2 = np.array([[359.73,359.73],[0.2,0.05]])
far3 = np.array([[359.61,359.61],[0.2,0.05]])
sgrc = np.array([[359.46,359.46],[-0.02,-0.08]])
stream = np.array([[359.65,359.65],[-0.29,-0.17]])
isolated = np.array([[359.14,359.14],[-0.34,0.0]])

In [ ]:
#IGNORE FOR NOW
# Don't run this cell 
###OMG matplotlib with wcs changes all the things and I'm finding  this difficult. 
#Just doing the fitsfigure version, see next box!!!
# Delete this??
#Plot SMA mask and assign cloud names to the numbers
%matplotlib nbagg


n = 20
color = plt.cm.tab10(np.linspace(0.1,0.9,n))
#color

plt.rcParams.update({'font.size': 12}) #set fontsize

fig, ax = plt.subplots(figsize=(12, 8))
ax = plt.subplot(projection=cmzoomWCS)

#fig.recenter(0.5,0.0, width=2.8, height=0.7)

cax = ax.imshow(mask)#, cmap='viridis')
ax.set_xlabel('Galactic Longitude')
ax.set_ylabel('Galactic Latitude')

#for i in np.arange(0,29): # Loop through each mask
#    sma_noise[i] = np.absolute(np.nanmedian(residual_pix[mask==i+1])) # calculate average noise and take absolute value so it's positive   
lon = ax.coords['glon']
lat = ax.coords['glat']
lon.set_major_formatter('d.dd')
lat.set_major_formatter('d.dd')
#lon.set_ticks(number=10)
lon.set_ticks(spacing=0.25 * u.degree)

#color='black'
alpha=0.65
# From CMZoom_coverage.ipynb

#fig.show_lines(line_list, color='black',alpha=0.65)
#ax.show_lines(line_list, color='black',alpha=0.65)

#Invert y axis (not sure why it's upside down)
plt.gca().invert_yaxis()

cbar = fig.colorbar(cax, orientation='horizontal') 
cbar.set_label('Mask Number') # to have label on bottom of colorbar

ax.invert_yaxis()
plt.show()
#plt.savefig(pathfig+'CMZoom_mask.pdf',
#            format='pdf', dpi=500, bbox_inches='tight')#, pad_inches=0.1)

In [ ]:
#IGNORE FOR NOW
#  From Betsy, this works if it's non-integer data (more complicated than needed)!
# use above for simplicity !!! (for now)


plot_ratio = np.copy(mask) #Make a copy of the mask array to drop values of the SMA/Bolocam ratio into
plot_ratio = 0.0*plot_ratio #Initialize it

S_flux=np.zeros(29) #Initialize other variables
B_flux=np.zeros(29)
Mass=np.zeros(29)
Area=np.zeros(29)
Column=np.zeros(29)

for i in np.arange(0,29): # Loop through each mask
    S_flux[i] = np.nansum(sma_pix[(mask>i)*(mask<=i+1)*(sma_pix>0)]) # Sum SMA above 0
    B_flux[i] = np.nansum(bgps_pix[(mask>i)*(mask<=i+1)*(bgps_pix>0)]) # Sum bolocam above 0
    Column[i] = np.nanmean(nh2[(mask>i)*(mask<=i+1)]) # Grab average column density
    plot_ratio[(mask>i)*(mask<=i+1)] = (S_flux[i]/B_flux[i]) #Fill each mask with the ratio of SMA to Bolocam
    
plt.imshow(plot_ratio)
plt.show()


In [ ]:
###IGNORE FOR NOW - WANT TO MAKE NEW MASK, DON"T THINK I CAN!!!
pathtemp='/Users/battersby/Dropbox/CMZoom_Data/continuum_images/Final_continuum_images/CB_TEMPORARY/'
from os import listdir
from os.path import isfile, join

filelist = [f for f in listdir(pathtemp) if isfile(join(pathtemp, f))]

num_mask = np.copy(cmzoom)

f=filelist[0]
hdulist = fits.open(pathtemp+f)
data=hdulist[0].data
hdr=hdulist[0].header

w = wcs.WCS(hdulist[0].header)

#pixcrd = np.array([[0, 0], [24, 38], [45, 98]], dtype=np.float64)
pixcrd = np.where(~np.isnan(data))

#world = w.wcs_pix2world(pixcrd, 0)
world = w.skycoord_to_pixel(pixcrd,w,origin=0)
print(world)


In [ ]:
###IGNORE FOR NOW - WANT TO MAKE NEW MASK, DON"T THINK I CAN!!!
# Make new Mask
num_mask = np.copy(cmzoom)

for f in filelist: 
    print(f)
    hdulist = fits.open(path+f)
    data=hdulist[0].data
    hdr=hdulist[0].header
    
    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 0-based (Numpy-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 0)
    print(world)

    # Convert the same coordinates back to pixel coordinates.
    pixcrd2 = w.wcs_world2pix(world, 0)
    print(pixcrd2)

    
    #Find the central coordinates and object name based on file name
    coords=[float(s) for s in re.findall(r'-?\d+\.?\d*', f)]
    negative = [s for s in re.findall(r'\-', str(coords[1]))]
    if negative == ['-']:
        obj_name = 'G'+str(coords[0])+str(coords[1])
        cloudnames.append(obj_name)
    else:
        obj_name = 'G'+str(coords[0])+'+'+str(coords[1])
        cloudnames.append(obj_name)
    
  


